# Supervised Classification of Landsat data

In [66]:
#pip install geemap
import ee
import geemap

# Creating Interactive map

In [5]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# Load LANDSAT data

In [30]:
point = ee.Geometry.Point([78, 28.7538])


image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(point).filterDate('2019-01-01', '2019-12-31').sort('CLOUD_COVER').first().select('B[1-7]')

band = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 8)
Map.addLayer(image, band, "Landsat-8")

In [10]:
Map.addLayerControl() 
Map

Map(bottom=27599.0, center=[28.753799999999995, 78], controls=(WidgetControl(options=['position', 'transparent…

# View image properties

In [31]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2019-04-28'

In [32]:

image.get('CLOUD_COVER').getInfo()

0

# Add region of interest

In [33]:
region = Map.user_roi
region = ee.Geometry.Rectangle([77.3836429942414, 28.72257092202146, 77.58002360947577, 28.597849875883103])
region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)


# Add labels using Copernicus Global Land Cover Layers: CGLS-LC100 collection 3

In [45]:
train = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification').clip(image.geometry())
Map.addLayer(train, {}, "Land Cover")
Map.setCenter(78.0, 28.4, 10)
Map


Map(bottom=55044.0, center=[28.4, 78.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

In [47]:
# Make the training dataset.
points = train.sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)

In [48]:

print(points.size().getInfo())

5000


In [49]:

print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [78.5156026010813, 28.45671896942989]}, 'id': '0', 'properties': {'discrete_classification': 40}}


# Train the classifier

In [54]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'discrete_classification'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': ['discrete_classification'],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [55]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 869, 'B2': 1050, 'B3': 1583, 'B4': 2001, 'B5': 2890, 'B6': 3643, 'B7': 3379, 'discrete_classification': 40}}


# Classify the image

In [65]:


# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)


Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

Map(bottom=871522.0, center=[29.105676541588767, 77.75436401367189], controls=(WidgetControl(options=['positio…